In [1]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import scipy
import torch
from torch.utils.data import DataLoader, ConcatDataset

os.chdir('..')
from data_processing.datasets import PigletDataset
from neuralnet.model import SpectraMLP
from utils import beerlamb_multi, beerlamb_multi_batch
import data_processing.preprocessing as preprocessing
from config import left_cut, right_cut

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
path_to_data = "dataset/piglet_diffs"
spectra_dataset = PigletDataset(path_to_data, range=(511, 512))
n = 4
spectra_datasets = ConcatDataset(n * [spectra_dataset])
dl = DataLoader(spectra_datasets, batch_size=n*len(spectra_dataset), shuffle=False)
inputs, targets = next(iter(dl))
print(len(spectra_datasets))

115624


In [3]:
img = scipy.io.loadmat('dataset/miniCYRIL-Piglet-Data/LWP511/LWP511_Ws_26Jul_2017_16   8.mat')
wavelengths = img['wavelengths'].astype(float)
idx = (wavelengths >= left_cut) & (wavelengths <= right_cut)
wavelengths = wavelengths[idx]

molecules, x = preprocessing.read_molecules(left_cut, right_cut, wavelengths)
molecules = molecules[:4]

In [4]:
def test(targets):
    start = time.time()
    preds = model(inputs.to(device).float()).cpu()
    duration = time.time() - start
    
    preds = torch.squeeze(preds)
    preds = torch.swapaxes(preds, 0, 1).detach()
        
    err = 0
    spec1 = np.asarray(np.log(beerlamb_multi_batch(molecules, x, preds, left_cut)))
    spec2 = np.asarray(np.log(beerlamb_multi_batch(molecules, x, torch.swapaxes(targets, 0, 1), left_cut)))
    err += np.mean(np.abs(spec1 - spec2))
    return duration, err

In [5]:
import pandas as pd

results_list = []
folders = os.listdir("results")

for folder in folders:
    layer, size = folder.split("_")

    model = SpectraMLP(4, n_layers=int(layer), layer_width=int(size))
    checkpoints = torch.load('results/{}/best_model.pth'.format(folder), map_location=torch.device('cpu'))
    model.load_state_dict(checkpoints)
    model.to(device)
    model.eval()

    duration, error = test(targets)

    results_dict = {
        'Folder': folder,
        'N Params': sum(p.numel() for p in model.parameters()),
        'Duration': round(duration, 4),
        'Error': round(error, 4)}
    results_list.append(results_dict)

df = pd.DataFrame(results_list)
#df.to_csv('results/results_summary.csv', index=False)

/Users/boeykaizhe/opt/anaconda3/envs/i2dl/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
df.sort_values(by='Error')

,Folder,N Params,Duration,Error
1,4_256,253444,0.5428,0.0127
3,2_128,44548,0.1218,0.0127
7,3_256,187652,0.3742,0.0127
6,4_128,77572,0.2187,0.0128
12,2_1024,1273860,2.6591,0.0128
5,2_256,121860,0.2722,0.0129
8,1_1024,224260,0.4293,0.0129
2,3_128,61060,0.2365,0.0132
0,1_128,28036,0.1607,0.0135
11,2_512,374788,0.6810,0.0135
